In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
rohanrao_formula_1_world_championship_1950_2020_path = kagglehub.dataset_download('rohanrao/formula-1-world-championship-1950-2020')

print('Data source import complete.')


Data source import complete.


The high speeds and the brake neck overtakes at some of the most notorious corners in racing. The sheer adrenaline of watching someone extract every single unit of the power from the engine and the level of tech and engineering that is poured into these cars makes F1 a brilliant sport. The Williams of 1992, with which Nigel Mansel swept the competition apart, was primarily due to the car maker inventing a groundbreaking suspension technology that would adapt based on different road conditions. From intricate technologies like active suspension to the unsophisticated nature of rear-view mirrors and steering wheel buttons, F1's engineering has enabled us to experience some of these in our road cars, making them a lot of efficient, safer and faster.

The following is an explorative analysis of F1 to break down some historical data on circuits, drivers and racing data to understand what makes them the best or the worst.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import datetime

### Reading the csv files

In [7]:
circuits = pd.read_csv("/kaggle/input/formula-1-world-championship-1950-2020/circuits.csv")
laptimes = pd.read_csv("/kaggle/input/formula-1-world-championship-1950-2020/lap_times.csv")
pitstops = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/pit_stops.csv')
seasons = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/seasons.csv',parse_dates=['year'])
status = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/status.csv')

In [8]:
constructor_standings = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructor_standings.csv')
constructors = pd.read_csv("/kaggle/input/formula-1-world-championship-1950-2020/constructors.csv")
driver_standings = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/driver_standings.csv')
drivers = pd.read_csv("/kaggle/input/formula-1-world-championship-1950-2020/drivers.csv")

In [9]:
races = pd.read_csv("/kaggle/input/formula-1-world-championship-1950-2020/races.csv",parse_dates=['year'])
constructor_results = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructor_results.csv')
results = pd.read_csv("/kaggle/input/formula-1-world-championship-1950-2020/results.csv")
qualifying = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/qualifying.csv')

### Most successful constructors

Many well-known, established names have competed at the highest level of racing with little success. A good reason for this is that F1 is not the best use of value creation, according to car makers, as the costs and returns associated with running an F1 team are close to breaking even at best. Good old finance says that any investment's return on equity being less than the cost of capital is a venture that destroys value. But this might not be entirely true in all cases as F1 provides a great platform to advertise and market the respective carmaker's brand; it's a cost that could be incurred to increase the top-line growth. Let's look at two charts to understand which constructors have consistently put in the money and engineering prowess to stay at the top. i.e. if you're a mega fan of the sport, you already know the answer.

In [10]:
#merging the constructors dataframe with race results

team = constructors.merge(results,on='constructorId',how = 'left')

In [11]:
#extracting the columns needed and grouping it by constructor name, extracting the total races entered

best = team[['name','points','raceId']]
best = best.groupby('name')['raceId'].nunique().sort_values(ascending=False).reset_index(name = 'races')
best = best[best['races'] >= 100]
best.head()

,name,races
0,Ferrari,1100
1,McLaren,929
2,Williams,843
3,Tyrrell,433
4,Sauber,419


In [12]:
#building a formula to calculate points per race

func = lambda x: x.points.sum()/x.raceId.nunique()
data = team[team['name'].isin(best.name)].groupby('name').apply(func).sort_values(ascending=False).reset_index(name = 'points_per_race')
data.head(10)

<ipython-input-12-db284b297c48>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = team[team['name'].isin(best.name)].groupby('name').apply(func).sort_values(ascending=False).reset_index(name = 'points_per_race')


,name,points_per_race
0,Mercedes,24.386877
1,Red Bull,19.474619
2,Ferrari,10.082973
3,McLaren,7.559203
4,Force India,5.179245
5,Renault,4.409429
6,Williams,4.319098
7,Benetton,3.313462
8,BRM,2.584135
9,Team Lotus,2.518987


In [13]:
#plotting the results

fig = go.Figure(
    data=[go.Bar(x = data.name, y=data['points_per_race'])],
    layout_title_text="Constructor's Points per Race"

)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.show()

Mercedes and Redbull have been highly consistent in the past decade, reflecting their points per race stat. On the contrary, Ferrari is yet to win a world championship since 2008. An interesting observation is Force India; considering it was a small-scale budget team compared to the giants like Mercedes and Ferrari, it did fantastic work on the track with an average of five points per race.

In [14]:
#calculating historic overall points of top 10 constructors

historic_points = team.groupby('name').agg({'points':'sum'}).sort_values('points',ascending=False).reset_index().head(10)
historic_points

,name,points
0,Ferrari,11091.27
1,Mercedes,7730.64
2,Red Bull,7673.00
3,McLaren,7022.50
4,Williams,3641.00
5,Renault,1777.00
6,Force India,1098.00
7,Team Lotus,995.00
8,Benetton,861.50
9,Tyrrell,711.00


In [15]:
#plotting a bar chart

fig = go.Figure(
    data=[go.Bar(x = historic_points.name, y=historic_points['points'])],
    layout_title_text="Constructor's Historic Points"
)
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.show()

The chart's most impressive is Mercedes AMG Petronas being second, considering they entered Formula in 2010. They've covered two-thirds of Ferrari's historical points in twelve years, an astonishing feat.

### Do higher altitude circuits cause more engine failures?

In higher altitudes, air density is thinner; therefore, less air passes through the radiators and intake valves to cool down the brakes and engine. Also, engines need oxygen to instantiate combustion, a lack of which leads to performance loss in a car. The primary data points for such scenarios are overheating of  transmission and engine components. Autódromo Hermanos Rodríguez, the Mexican GP, is situated at 2227 meters above sea level, making it the highest track on the F1 calendar by a fair margin. It's well known among the F1 community the sheer problems the circuit produces. The engineers must redesign and rethink many aspects of approaching race week as performance loss is expected to come into Friday practice sessions.

In [19]:
#merging circuits, races, results and race status dataframes

df = circuits.merge(races,how='left',left_on = 'circuitId',right_on = 'circuitId')
df2 = df.merge(results,how='left',on='raceId')
status_df = df2.merge(status,how='inner',left_on = 'statusId',right_on= 'statusId')

In [20]:
#cosmetic changes: dropping columns and renaming

status_df.drop(['name_y','url_y','url_x','time_y'],axis=1,inplace=True)
status_df.rename(columns={'name_x':'name','time_x':'time'},inplace=True)

In [21]:
#including rows with issues correlated with thin air in higher altitudes, setting the year to last 7 to include Mexico GP

altitude = status_df[status_df['status'].isin(['Transmission','Engine','Overheating'])]
altitude = altitude[altitude['year'] >= pd.to_datetime('2015-01-01')]
altitude.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,raceId,year,...,positionOrder,points,laps,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,status
430,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,926,2015-01-01,...,13,0.0,32,\N,30,13,1:34.295,202.458,5,Engine
452,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,948,2016-01-01,...,17,0.0,38,\N,15,21,1:33.892,203.327,5,Engine
453,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,948,2016-01-01,...,18,0.0,21,\N,21,5,1:30.701,210.480,5,Engine
495,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,989,2018-01-01,...,18,0.0,13,\N,13,18,1:30.649,210.601,5,Engine
517,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1010,2019-01-01,...,20,0.0,9,\N,9,20,1:30.899,210.022,5,Engine


In [22]:
#grouping by track name and altitude and renaming columns

circuit_altitudes = altitude.groupby(['name','alt'])['status'].count().sort_values(ascending = False).reset_index().head(10)
circuit_altitudes.rename(columns={'status':'engine & transmission failures'},inplace=True)
circuit_altitudes

,name,alt,engine & transmission failures
0,Autodromo Nazionale di Monza,162,9
1,Autódromo Hermanos Rodríguez,2227,8
2,Albert Park Grand Prix Circuit,10,7
3,Autódromo José Carlos Pace,785,6
4,Bahrain International Circuit,7,6
5,Yas Marina Circuit,3,6
6,Red Bull Ring,678,6
7,Marina Bay Street Circuit,18,6
8,Circuit Gilles Villeneuve,13,4
9,Circuit de Spa-Francorchamps,401,4


In [23]:
#plotting a bubble chart: bigger the size of the bubble, higher the altitude

df = circuit_altitudes

fig = px.scatter(df, x="alt", y="engine & transmission failures",
         size="alt", color="name",
                  log_x=True, size_max= 80)
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

As expected, the Mexican circuit poses the highest overheating issues, due to which cars had to retire, followed by the Red Bull Ring. But Bahrain GP, close to sea level, sees the same amount of trouble. This may be due to high track temperatures because of the geographic location. These are mere observations without any statistical tests to prove my hypothesis. Understanding why certain things happen before applying statistical modelling is essential. Mapping track temperatures and altitude to the race status of every driver using supervised algorithms or simple linear regression techniques could enable the possibility of causality.

### The case for the best F1 drivers  

Seventy-two years of F1 history has been 34 different champions. Cars over the years been tremendous changes. From the early 2000s V10 screaming notes to the exceptionally engineered and safer V6, the sport has considerably changed in the last 20 years. A few drivers on the grid witnessed their best in one era before giving way to younger drivers to take over. Let's look at the first chart that depicts historic champion distribution by nation.

#### Distribution by Geography

In [24]:
# grouping by nationality, counting the driver and plotting a pie chart

driver_nationality = drivers.groupby('nationality')['nationality'].count().sort_values(ascending = False).reset_index(name = 'number of drivers')
fig = go.Figure(data=[go.Pie(labels=driver_nationality.nationality.head(10), values=driver_nationality['number of drivers'])])
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    title="Historical Driver Nationality Distribution since 1950")
fig.show()

In [26]:
#merging drivers, driver standings and race data

driver_position = drivers.merge(driver_standings,left_on='driverId',right_on='driverId',how = 'left')
driver_position = driver_position.merge(races,on = 'raceId',how = 'left')

In [66]:
#grouping by nationality year and surname to get the max points achieved every season

champions = driver_position.groupby(['nationality','year','surname'])[['points','wins']
                                            ].max().sort_values('points',ascending = False).reset_index()
champions.drop_duplicates(subset=['year'], inplace=True)

In [68]:
# counting the number of times a nation ended in P1 and plotting a pie chart

champion_nations = champions['nationality'].value_counts().to_frame(name='count')
fig = go.Figure(data=[go.Pie(labels=champion_nations.index, values=champion_nations['count'])])
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    title="Distribution of Historic Champions by Nation")
fig.show()


To understand why F1 saw the most British drivers and champions, we'll have to transcend back to WWII and the prevailing aerial duels over the British channel against the Germans. Due to the constant aerial warfare, the British were forced to build massive airfields to defend against the Germans. Post-WWII and the fall of Nazi Germany made these airfields utterly unserviceable until a band of British motor enthusiasts decided to turn these airfields into creative race tracks. Soon enough, this attracted race car drivers and engineers who worked on complex fighter jet engines in the war to build the best race cars and test them on the now-converted race tracks. One of the airfields went on to become the "Mecca" of racing, The Silverstone Circuit. Over the years, the influx of racing talent to Britain has seen many F1 teams set up their headquarters in the region. 6 out of 10  constructors in 2022 have their offices in the United Kingdom.

In [30]:
#grouping by nationality year and surname to get the max points achieved every season and dropping year duplicates

champion_drivers = driver_position.groupby(['nationality','year','surname'])[['points','wins']
                                            ].max().sort_values('points',ascending = False).reset_index()
champion_drivers.drop_duplicates(subset=['year'], inplace=True)

#grouping by nationality and counting the surname of drivers

final = champion_drivers.groupby('nationality')['surname'].nunique().reset_index(name = 'champions').sort_values(
    by='champions',ascending = False)

#merging both the datasets and creating a column to calculate the ratio

ratios = final.merge(driver_nationality,on='nationality',how='inner')
ratios['perc_winners'] = (ratios.champions/ratios['number of drivers']*100).round(2)
ratios = ratios.sort_values('perc_winners',ascending = False)
ratios.head(5)

,nationality,champions,number of drivers,perc_winners
2,Finnish,3,9,33.33
5,Austrian,2,15,13.33
4,Australian,2,19,10.53
12,New Zealander,1,10,10.00
1,Brazilian,3,32,9.38


In [31]:
#creating a bar chart

df = ratios
fig = px.bar(df, x='nationality', y='perc_winners',
         hover_data=['champions','number of drivers'], color='number of drivers',
         height=400)
fig.update_traces(textfont_size=20,
              marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    title="Champions from a nation with respect to total drivers from the nation")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

Can we put the success of the Finnish in F1 down to chance and luck with just nine drivers entering the grid, or is it much more than that? If we start digging deeper, we realise what makes a country of just five million people such good racing drivers. One reason could be the exposure to automobiles at a young age because the skillset required to drive in the harsh cold terrains of Finnish lands needs to be introduced as early as possible. This was also quoted by Kimi Raikonnen (The last Ferrari world champion and Finnish driver), "Our roads and long winters. You really have to be a good driver to survive in Finland. It is always slippery and bumpy." The second reason is Finland has a lively racing culture amongst the old and the young, with the country having nearly 20 official carting tracks, one of the highest per capita in the world. Motor racing requires a level of composure that's uncalled for in regular day-to-day instances, mainly due to blasting down the straights and curves at over 250kmph. The Finnish seem to have a knack for this in their blood; they call it Sisu. In English, it loosely translates into a stoic sense of determination and purpose while accepting the outcome without judgement. They call Kimi Raikonnen "The Iceman" for no reason; the Finnish drivers are racers of few words. When Kimi's car caught fire in the 2006 Monaco GP while fighting for first place, he simply strolled out of the car and onto his yacht and watched the race while having beers and champagne.

#### Most wins by a driver in a single season

In [71]:
#merging driver data, their standings and race data

driver_position = drivers.merge(driver_standings,left_on='driverId',right_on='driverId',how = 'left')
driver_position = driver_position.merge(races,on = 'raceId',how = 'left')

In [72]:
#filtering the dataset to include only where the position is 1 and grouping by name, year and extracting the max wins

positions = driver_position[driver_position['position'] == 1].groupby(
    ['surname','year'])['wins'].max().sort_values(ascending=False).reset_index(name = 'Wins')
positions.head(20)
positions.year = positions.year.dt.year
positions.rename(columns={'surname':'name'},inplace=True)
positions.Wins = positions.Wins.astype('int64')

positions.head(20)

,name,year,Wins
0,Verstappen,2023,19
1,Verstappen,2022,15
2,Vettel,2013,13
3,Schumacher,2004,13
4,Hamilton,2020,11
5,Hamilton,2019,11
6,Hamilton,2018,11
7,Hamilton,2014,11
8,Schumacher,2002,11
9,Vettel,2011,11


In [73]:
#plotting a bubble chart

fig = px.scatter(positions.head(30), x="year", y="Wins", color="name",
                 title="Most wins by a driver in a single season",size = 'Wins')
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.show()

Vettel and Schumacher have a combined record for most wins in a single season. Both were utterly dominant in the years 2004 and 2013, respectively. The case for the best F1 driver is strong for both Schumacher (blue) and Hamilton (green), and it wouldn't be wrong to choose either as the best F1 driver of all time, but does the number of championships all that matters? Some argue that the calibre of racers the champion faced makes them the best, Senna vs Alain Prost, Hamilton vs Rosberg. Well, the best-ever talk is subjective in most modern day sports, so I'll leave that up to you to decide. I've written a python script (link below the blog) that classifies a season as one of the greatest rivalries or the season being meh based on the year input. Maybe this way the greatest ever could be decided according to you. I'll leave one below as an example.

#### Most competitive seasons by points difference

In [74]:
competition = driver_position[driver_position['year'].dt.year == 1991
               ].groupby(['surname','year']).points.max().sort_values(ascending = False).reset_index().head(5)
competition.year = competition.year.dt.year
competition.iloc[0:2,0]

,surname
0,Senna
1,Mansell


In [78]:
def rivalry(x):
    competition = driver_position[driver_position['year'].dt.year == x
               ].groupby(['surname','year']).points.max().sort_values(ascending = False).reset_index().head(5)
    competition.year = competition.year.dt.year

    h = driver_position.merge(circuits,left_on='circuitId',right_on='circuitId',how = 'left')
    h.rename(columns={'name_y':'circuit_name'},inplace=True)
    viz = h.loc[:,['date','year','circuit_name','surname','points','wins']]

    viz.dropna(inplace = True)

    viz.points = viz.points.astype('int64')
    viz.wins = viz.wins.astype('int64')
    viz.year = viz.year.dt.year
    viz.date = pd.to_datetime(viz.date)

    top_five = viz[viz.loc[:,'year'] == x]
    top_five = top_five.groupby(['surname'])[['points','wins']].max().sort_values('points',ascending = False).head(6).reset_index()

    duo = competition.iloc[0:2]
    if competition.iloc[0,2] - competition.iloc[1,2] <= 10:
        print('\033[1m' + 'A rivalry in the history books!')
    elif competition.iloc[0,2] - competition.iloc[1,2] <= 20:
        print('\033[1m' + 'Spicy!')
    elif competition.iloc[0,2] - competition.iloc[1,2] < 30:
        print('\033[1m' + 'Meh!')
    elif competition.iloc[0,2] - competition.iloc[1,2] >= 30:
        print('\033[1m' + 'Snore Fest!')


    df = top_five
    fig = px.bar(df, x='surname', y='points',
             hover_data=['wins'], color='points',
            height=400,color_continuous_scale= 'turbo')
    fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)

    print('----------------------------------')
    x = fig.show()
    return x

**PLAY AROUND WITH THE YEAR BELOW**

In [80]:
rivalry(2022)

Snore Fest!
----------------------------------


### Who has the fastest lap time in every circuit?

In [85]:
#merging and extraction of important columns


# Merge all datasets
fast = circuits.merge(races, left_on='circuitId', right_on='circuitId', how='left')
fast = fast.merge(results, on='raceId', how='left')
fast = fast.merge(drivers, left_on='driverId', right_on='driverId', how='inner')

# Rename and extract only necessary columns
fast.rename(columns={'name_x': 'circuit_name'}, inplace=True)
fast = fast[['circuit_name', 'country', 'surname', 'fastestLapTime', 'nationality', 'year']]

# Drop rows where 'fastestLapTime' is NaN or '\N'
fast = fast.dropna(subset=['fastestLapTime'])
fast = fast[fast['fastestLapTime'] != '\\N']

# Convert fastestLapTime to seconds (safe approach)
def convert_time(x):
    try:
        m, s = x.split(':')
        return float(m) * 60 + float(s)
    except:
        return None

fast['fastestLapTime_seconds'] = fast['fastestLapTime'].apply(convert_time)

# Drop any rows where conversion failed
fast = fast.dropna(subset=['fastestLapTime_seconds'])

# Optional: reset index for cleaner further use
fast.reset_index(drop=True, inplace=True)

In [86]:
fast['fastest_recorded_lap'] = fast.groupby(['circuit_name'])['fastestLapTime'].transform('min')
fastest = fast[fast['fastest_recorded_lap']==fast['fastestLapTime']].sort_values('country').reset_index(col_level = 1)
fastest.drop(['index','fastest_recorded_lap'],inplace=True,axis = 1)
fastest['year'] = fastest.year.dt.year
fastest.head()

,circuit_name,country,surname,fastestLapTime,nationality,year,fastestLapTime_seconds
0,Albert Park Grand Prix Circuit,Australia,Leclerc,1:19.813,Monegasque,2024,79.813
1,Red Bull Ring,Austria,Sainz,1:05.619,Spanish,2020,65.619
2,Baku City Circuit,Azerbaijan,Leclerc,1:43.009,Monegasque,2019,103.009
3,Bahrain International Circuit,Bahrain,Russell,0:55.404,British,2020,55.404
4,Circuit de Spa-Francorchamps,Belgium,Pérez,1:44.701,Mexican,2024,104.701


In [87]:
fastest_viz = fastest.surname.value_counts().rename_axis('driver').reset_index(name= 'fastest laps')

df = fastest_viz
fig = px.bar(df, x='driver', y='fastest laps',
         hover_data=['fastest laps'], color='fastest laps',
        height=400,color_continuous_scale= 'Blues')
fig.update_layout(
    title="Drivers with the most fastest ever laps")
fig.update_traces(textfont_size=20,
              marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

####  

### Qualifying Pole = Race Pole?

In F1, finishing at the pole on Saturday often does not guarantee the same on race day. What makes the mark of a great driver? Finishing pole every time he starts at the pole? A perfect ratio of 1? Or someone who's won more races than qualifying wins?

##### Calculating grid pole positions

In [88]:
#merging the (1) two dataframes [drivers and qualifying] needed and (2) performing aggregation function

driver_quali = results.merge(drivers,left_on = 'driverId',right_on='driverId',how = 'left')
driver_quali['full_name'] = driver_quali['forename'] + ' ' + driver_quali['surname']
driver_quali = driver_quali[['full_name','grid','position']]


quali_wins = driver_quali[driver_quali['grid'] == 1].groupby('full_name')['grid'].sum().reset_index(
                                        name = 'grid poles').sort_values('grid poles',ascending = False)

#resetting index numbers

quali_wins = quali_wins.reset_index(col_level=0)
quali_wins.drop('index',axis = 1,inplace = True)
quali_wins.head(20)

,full_name,grid poles
0,Lewis Hamilton,104
1,Michael Schumacher,68
2,Ayrton Senna,65
3,Sebastian Vettel,57
4,Max Verstappen,40
5,Jim Clark,34
6,Alain Prost,33
7,Nigel Mansell,32
8,Nico Rosberg,30
9,Juan Fangio,29


##### Calculating race wins  

In [90]:
#merging (1) dataframes [drivers, results], (2) creating a full name column and (3) selecting the important columns


# 1. Merge drivers and results
race_wins = drivers.merge(results, on='driverId', how='left')

# 2. Create full_name
race_wins['full_name'] = race_wins['forename'] + ' ' + race_wins['surname']

# 3. Keep only the columns we need
race_wins = race_wins[['full_name', 'position']]

# 4. Convert position to numeric (invalid parsing becomes NaN)
race_wins['position'] = pd.to_numeric(race_wins['position'], errors='coerce')

# 5. Filter for wins (position == 1), group and count
highest_rw = (
    race_wins[race_wins['position'] == 1]
    .groupby('full_name', as_index=False)
    .agg(wins=('position', 'count'))
    .sort_values('wins', ascending=False)
)

# 6. View the top 20 winners
print(highest_rw.head(20))


              full_name  wins
66       Lewis Hamilton   105
75   Michael Schumacher    91
74       Max Verstappen    63
106    Sebastian Vettel    53
0           Alain Prost    51
4          Ayrton Senna    41
24      Fernando Alonso    32
81        Nigel Mansell    31
37       Jackie Stewart    27
82           Niki Lauda    25
47            Jim Clark    25
60          Juan Fangio    24
79        Nelson Piquet    23
80         Nico Rosberg    23
13           Damon Hill    22
63       Kimi Räikkönen    21
77        Mika Häkkinen    20
108       Stirling Moss    16
46        Jenson Button    15
21   Emerson Fittipaldi    14


##### Calculating RacePole / GridPole

In [92]:
# Merging race wins with grid pole positions
racexpole = highest_rw.merge(quali_wins, on='full_name', how='left')

# Drop null values and filter for minimum 10 poles
racexpole = racexpole.dropna(subset=['grid poles'])
racexpole = racexpole[racexpole['grid poles'] > 10]

# Rename for clarity (race poles already exists as 'wins')
racexpole.rename(columns={'wins': 'race poles'}, inplace=True)

# Convert grid poles to int just in case
racexpole['grid poles'] = racexpole['grid poles'].astype(int)

# Calculate ratio of wins from pole
racexpole['racexgrid'] = (racexpole['race poles'] / racexpole['grid poles']).round(2)

# Sort by this ratio
racexpole = racexpole.sort_values('racexgrid', ascending=False).reset_index(drop=True)

# Display top 15
print(racexpole.head(15))


             full_name  race poles  grid poles  racexgrid
0       Jackie Stewart          27          17       1.59
1       Max Verstappen          63          40       1.58
2          Alain Prost          51          33       1.55
3      Fernando Alonso          32          22       1.45
4   Michael Schumacher          91          68       1.34
5       Kimi Räikkönen          21          18       1.17
6           Damon Hill          22          20       1.10
7      David Coulthard          13          12       1.08
8          Graham Hill          14          13       1.08
9         Jack Brabham          14          13       1.08
10          Niki Lauda          25          24       1.04
11      Lewis Hamilton         105         104       1.01
12       Nigel Mansell          31          32       0.97
13       Nelson Piquet          23          24       0.96
14       Stirling Moss          16          17       0.94


In [93]:
df = racexpole.head(12)
fig = px.bar(df, x='full_name', y='racexgrid',
         hover_data=['race poles','grid poles'], color='racexgrid',
        height=400,color_continuous_scale= 'gray')
fig.update_layout(
    title="Race Poles/Grid Poles Ratio")
fig.update_traces(textfont_size=20,
              marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

The above chart indicates Max Verstappen has more wins (26) than pole positions (16). His innate ability to thrive under pressure, as we witnessed in the last lap of the 2021 season and his aggressive driving stance helps him win races even if he isn't the best in qualifying. This is also a case for why Schumacher would be considered the best ever because he's extracted those race points and championships from more challenging grid positions than Lewis Hamilton.

### Worst tracks based on overtaking action

There are always a few tracks in the calendar year that we know will be a snoozefest. Poor racing and overtake action happen on circuits, with most of the design and racing happening in straight lines and nothing else. To check for the least overtaking action, I counted the number of times the positions of the cars remained unchanged from the start till the end of the race. There's always a possibility of overtaking and the driver returning to his initial position. Although this method of rating tracks is not perfect, it'll give us a reasonable glimpse of potentially bland circuits.

In [95]:
tracks = circuits.merge(races, left_on='circuitId',right_on='circuitId',how = 'inner')
tracks = tracks.merge(results,on = 'raceId',how = 'left')
tracks = tracks[['name_x','circuitId','driverId','position','grid','raceId','year']]
tracks.rename(columns={'name_x':'circuit'},inplace=True)
tracks.dropna(inplace = True)


tracks = tracks.loc[(tracks['year'] >= '2010-01-01')]
tracks

,circuit,circuitId,driverId,position,grid,raceId,year
302,Albert Park Grand Prix Circuit,1,18,1,4,338,2010-01-01
303,Albert Park Grand Prix Circuit,1,9,2,9,338,2010-01-01
304,Albert Park Grand Prix Circuit,1,13,3,5,338,2010-01-01
305,Albert Park Grand Prix Circuit,1,4,4,3,338,2010-01-01
306,Albert Park Grand Prix Circuit,1,3,5,6,338,2010-01-01
...,...,...,...,...,...,...,...
26754,Miami International Autodrome,79,822,16,16,1126,2024-01-01
26755,Miami International Autodrome,79,840,17,11,1126,2024-01-01
26756,Miami International Autodrome,79,848,18,14,1126,2024-01-01
26757,Miami International Autodrome,79,825,19,18,1126,2024-01-01


In [101]:
tracks['position_status'] = np.where(tracks['position'] == tracks['grid'],1,0)

In [105]:
# Step 1: Make sure 'circuitId' is of the same type in both DataFrames
tracks['circuitId'] = tracks['circuitId'].astype(int)
races['circuitId'] = races['circuitId'].astype(int)

# Step 2: Filter races from 2010 onwards
races['year'] = pd.to_datetime(races['year'], errors='coerce')  # ensure proper date format
recent_races = races[races['year'] >= '2010-01-01']

# Step 3: Count total number of races per circuit
total_races = recent_races['circuitId'].value_counts().reset_index()
total_races.columns = ['circuitId', 'num_races']  # rename for clarity

# Step 4: Merge with tracks
tracks = tracks.merge(total_races, on='circuitId', how='left')

# Step 5: Done! View the result
print(tracks.head())



                          circuit  circuitId  driverId position  grid  raceId  \
0  Albert Park Grand Prix Circuit          1        18        1     4     338   
1  Albert Park Grand Prix Circuit          1         9        2     9     338   
2  Albert Park Grand Prix Circuit          1        13        3     5     338   
3  Albert Park Grand Prix Circuit          1         4        4     3     338   
4  Albert Park Grand Prix Circuit          1         3        5     6     338   

        year  position_status  num_races  
0 2010-01-01                0         13  
1 2010-01-01                0         13  
2 2010-01-01                0         13  
3 2010-01-01                0         13  
4 2010-01-01                0         13  


In [111]:
#pivot table to find the count of unchanged positions [marked numbers column 1 values]

# Step 1: Filter circuits with at least 5 races
filtered_tracks = tracks[tracks['num_races'] >= 5]

# Step 2: Keep only rows where position_status == 0 (unchanged)
unchanged_positions = filtered_tracks[filtered_tracks['position_status'] == 0]

# Step 3: Count unchanged positions per circuit
circuit_rating = unchanged_positions.groupby(['circuit', 'num_races']).size().reset_index(name='unchanged_positions')

# Step 4: Sort by most unchanged positions
circuit_rating = circuit_rating.sort_values(by='unchanged_positions', ascending=False).reset_index(drop=True)

# Step 5: Display result
print(circuit_rating.head(10))



                          circuit  num_races  unchanged_positions
0             Silverstone Circuit         16                  338
1    Circuit de Spa-Francorchamps         15                  318
2                     Hungaroring         15                  318
3  Circuit de Barcelona-Catalunya         15                  318
4    Autodromo Nazionale di Monza         15                  318
5              Yas Marina Circuit         15                  316
6   Bahrain International Circuit         15                  314
7               Circuit de Monaco         14                  298
8      Autódromo José Carlos Pace         14                  294
9                  Suzuka Circuit         13                  278


In [116]:
#checking for tracks that had the least amount of overtaking action
# Step 1: Filter tracks with at least 5 races
filtered = tracks[tracks['num_races'] >= 5]

# Step 2: Count total records per track
total_counts = filtered.groupby(['circuit', 'num_races']).size().reset_index(name='total_entries')

# Step 3: Count unchanged positions
unchanged_counts = filtered[filtered['position_status'] == 0].groupby(
    ['circuit', 'num_races']
).size().reset_index(name='unchanged_positions')

# Step 4: Merge both counts
circuit_rating = total_counts.merge(unchanged_counts, on=['circuit', 'num_races'], how='left')
circuit_rating['unchanged_positions'].fillna(0, inplace=True)

# Step 5: Calculate boring_score
circuit_rating['boring_score'] = (circuit_rating['unchanged_positions'] / circuit_rating['total_entries']).round(2)

# Step 6: Sort by boring_score (high to low = most boring)
circuit_rating = circuit_rating.sort_values('boring_score', ascending=False).reset_index(drop=True)

# Step 7: View top 10 most boring tracks
print(circuit_rating.head(10))


                          circuit  num_races  total_entries  \
0  Albert Park Grand Prix Circuit         13            275   
1    Autodromo Nazionale di Monza         15            318   
2    Autódromo Hermanos Rodríguez          9            182   
3      Autódromo José Carlos Pace         14            294   
4   Bahrain International Circuit         15            314   
5               Baku City Circuit          8            162   
6       Circuit Gilles Villeneuve         13            278   
7  Circuit de Barcelona-Catalunya         15            318   
8               Circuit de Monaco         14            298   
9    Circuit de Spa-Francorchamps         15            318   

   unchanged_positions  boring_score  
0                  275           1.0  
1                  318           1.0  
2                  182           1.0  
3                  294           1.0  
4                  314           1.0  
5                  162           1.0  
6                  278           

<ipython-input-116-535d281f767a>:15: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [118]:
import plotly.express as px

df = circuit_rating.head(10)  # Top 10 circuits with highest boring score

fig = px.bar(
    df,
    x='circuit',
    y='boring_score',
    hover_data=['num_races', 'unchanged_positions'],  # ✅ FIXED HERE
    color='num_races',
    height=400,
    color_continuous_scale='ice'
)

fig.update_layout(
    title="Worst Tracks to Overtake",
    xaxis_title="Circuit",
    yaxis_title="Boring Score"
)

fig.update_traces(
    textfont_size=20,
    marker=dict(line=dict(color='#000000', width=2))
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()


The data is filtered to include only observations after 2009. Monaco, considered an iconic track for the generational history of F1, makes it to this list. Monaco GP is poorly designed for modern-day F1 cars, which are much bigger than the race cars of the 1990s. The lack of room in the track makes it extremely hard for drivers to overtake on race day.